In [248]:
!conda install -c conda-forge folium=0.5.0

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


# Importing all the libraries required for the project

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from pandas import read_html

print('Libraries imported.')

Libraries imported.


### Using Pandas.read_html() Function to extract the wikitable from the website
 I found this approach to be easier to implement compared to using BeautifulSoup

In [4]:
page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(page, index_col=0, attrs={"class":"wikitable sortable"})
print("Extracted {num} wikitables".format(num=len(wikitable)))

Extracted 1 wikitables


In [5]:
df=wikitable[0]

In [6]:
df.head()

,1,2
0,,
Postcode,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village


In [7]:
df=df.drop(['Postcode'],axis=0)

In [8]:
df=df.reset_index()

Renaming the columns of the dataset

In [9]:
df=df.rename(index=str, columns={0: "Postcode", 1: "Borough",2:"Neighborhood"})

In [10]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df.shape

(289, 3)

#### Dropping the rows that have Borough values as Not Assigned and copying the Borough values to Neighborhood if Neighborhood is Not Assigned

In [12]:
for x in range(0,289) :
    i=str(x)
    if df.loc[i,'Borough'] == "Not assigned" :
        df=df.drop([i],axis=0)
    elif df.loc[i,'Neighborhood'] == "Not assigned" :
        df.loc[i,'Neighborhood'] = df.loc[i,'Borough']

In [13]:
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
df.shape

(212, 3)

We have 103 unique postcodes

In [15]:
unique = [] 
for x in df['Postcode']: 
    if x not in unique: 
        unique.append(x) 
print(len(unique))

103


In [16]:
df2=df

#### A Loop to merge neighborhoods 

In [17]:
postcode = unique
borough=[]
neighborhood=[]
ind=0
for x in postcode :
    length=len(df2[df2['Postcode'] == x].index)
    indexs=df2[df2['Postcode'] == x].index
    if length == 1 :
        borough.append(df.loc[str(indexs[0]),'Borough'])
        neighborhood.append(df.loc[str(indexs[0]),'Neighborhood'])
    else :
        borough.append(df.loc[str(indexs[0]),'Borough'])
        neighborhood.append(df2.loc[str(indexs[0]),'Neighborhood'])
        for i in indexs[1:]:
            pos=str(i)
            neighborhood[ind]=neighborhood[ind]+', '+df2.loc[pos,'Neighborhood']
    ind=ind+1

In [18]:
print(len(postcode),len(borough),len(neighborhood))

103 103 103


In [19]:
df2=pd.DataFrame({'Postcode':postcode,'Borough':borough,"Neighborhood":neighborhood})

DF2 is the Dataframe containing the merged neighborhoods

In [20]:
df2.head()

,Borough,Neighborhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park,Queen's Park,M7A


In [21]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  42.85 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  12.30 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  46.45 MB/s


In [22]:
import os

#### As I am using IBM Watson Studio, This is the way to import the geospacial coordinates csv file

In [23]:
os.getcwd()
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ad83cc30342842399f72f99a5b2a02ad = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='917biCoTTkXtSurdjW6PM6UJg_480wQJEMDIKCBnF_a2',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ad83cc30342842399f72f99a5b2a02ad.get_object(Bucket='ankushx27scapstone-donotdelete-pr-cbrmw1dxeteyxv',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
coord=df_data_1

In [25]:
lat=[]
lng=[]
for x in postcode :
    ind=coord[coord['Postal Code'] == x].index[0]
    lat.append(coord.loc[ind,'Latitude'])
    lng.append(coord.loc[ind,'Longitude'])
    

In [26]:
print(len(lat),len(lng))

103 103


In [27]:
ind=coord[coord['Postal Code'] == 'M3A'].index[0]
coord.loc[ind,'Latitude']

43.753258600000002

Cross checking if the copy operation worked

In [28]:
df2['Latitude']=lat

In [29]:
df2.head()

,Borough,Neighborhood,Postcode,Latitude
0,North York,Parkwoods,M3A,43.753259
1,North York,Victoria Village,M4A,43.725882
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518
4,Queen's Park,Queen's Park,M7A,43.662301


In [30]:
df2['Longitude']=lng

In [31]:
df2.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [32]:
neighborhoods = df2

# Performing the same analysis done as that on the New York Dataset

In [228]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Plotting the Toronto map using Folium

In [229]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [230]:
# @hidden_cell
CLIENT_ID = '1QKLV2GDIKOCAD1HFKRQDHYNSIVNQYA02RZEFQVJXT0QL5SK' # your Foursquare ID
CLIENT_SECRET = 'O53UIHGDY1ER23NUNWPYLRGVFZ1KJFZUJB4VOO5PHOJSXEXG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1QKLV2GDIKOCAD1HFKRQDHYNSIVNQYA02RZEFQVJXT0QL5SK
CLIENT_SECRET:O53UIHGDY1ER23NUNWPYLRGVFZ1KJFZUJB4VOO5PHOJSXEXG


In [231]:
neighborhoods.loc[0,'Neighborhood']

'Parkwoods'

In [232]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [233]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude
    )
#https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD


Obtaining the results from the FourSquare API

In [234]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9c6ffb9fb6b73b71c23fcb'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'primary': True,
      'shortName': 'Park'}],
    'hasPerk': False,
    'id': '4e8d9dcdd5fbbbb6b3003c7b',
    'location': {'address': 'Toronto',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 245,
     'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.751976046055574,
       'lng': -79.33214044722958}],
     'lat': 43.751976046055574,
     'lng': -79.33214044722958,
     'state': 'ON'},
    'name': 'Brookbanks Park',
    'referralId': 'v-1553756155'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/apartment_',
   

In [235]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [236]:
results['response']

{'confident': False,
 'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'primary': True,
     'shortName': 'Park'}],
   'hasPerk': False,
   'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'location': {'address': 'Toronto',
    'cc': 'CA',
    'city': 'Toronto',
    'country': 'Canada',
    'distance': 245,
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'state': 'ON'},
   'name': 'Brookbanks Park',
   'referralId': 'v-1553756155'},
  {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/apartment_',
      'suffix': '.png'},
     'id': '4d954b06a243a5684965b473',
     'name': 'Residential Building (Apart

In [237]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,categories
0,Brookbanks Park,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",43.751976,-79.332140,Park
1,15 Brookbanks,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",43.752266,-79.332322,Residential Building (Apartment / Condo)
2,29 Valleywoods Rd,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",43.753868,-79.331849,Residential Building (Apartment / Condo)
3,Corey's Hood,[],43.753638,-79.340201,None
4,Tastybite Fish & Chips,[],43.760336,-79.325309,None


In [238]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [239]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [240]:
print(toronto_venues.shape)
toronto_venues.head()

(2235, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Checking the number of venues obtained for each Neighborhood

In [241]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,58,58,58,58,58,58


In [242]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


Performing One Hot Encoding on the dataset

In [243]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [244]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()


,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [249]:
list1=toronto_onehot.columns.tolist()
list1.index('Neighborhood')

189

In [250]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[189]] + list(toronto_onehot.columns[:189])+list(toronto_onehot.columns[190:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Office,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [251]:
toronto_onehot.shape

(2235, 269)

In [252]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Office,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.01000

In [253]:
toronto_grouped.shape

(99, 269)

Displaying the top 5 venues for each Neighborhood

In [254]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1              Bar  0.04
2             Café  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
                venue  freq
0      Clothing Store  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3        Skating Rink  0.25
4  Mexican Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2          Pizza Place  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Alderwood, Long 

                        venue  freq
0               Grocery Store  0.50
1               Shopping Mall  0.25
2                        Bank  0.25
3                      Office  0.00
4  Modern European Restaurant  0.00


----East Birchmount Park, Ionview, Kennedy Park----
               venue  freq
0     Discount Store  0.17
1         Hobby Shop  0.17
2  Convenience Store  0.17
3        Bus Station  0.17
4   Department Store  0.17


----East Toronto----
                venue  freq
0   Convenience Store  0.33
1         Coffee Shop  0.33
2                Park  0.33
3              Office  0.00
4  Mexican Restaurant  0.00


----Emery, Humberlea----
                        venue  freq
0      Furniture / Home Store   0.5
1              Baseball Field   0.5
2                      Office   0.0
3          Mexican Restaurant   0.0
4  Modern European Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1  Fast Food Restaurant  0.10
2    

                  venue  freq
0  Fast Food Restaurant   1.0
1                 Motel   0.0
2                Market   0.0
3     Martial Arts Dojo   0.0
4        Massage Studio   0.0


----Runnymede, Swansea----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.08
2       Pizza Place  0.08
3          Tea Room  0.05
4  Sushi Restaurant  0.05


----Ryerson, Garden District----
                       venue  freq
0             Clothing Store  0.07
1                Coffee Shop  0.07
2             Cosmetics Shop  0.04
3                       Café  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                             venue  freq
0                       Playground   0.5
1                Convenience Store   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2           Hotel

In [255]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [256]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Thai Restaurant,Steakhouse,Café,American Restaurant,Bakery,Hotel,Sushi Restaurant,Asian Restaurant
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,German Restaurant,Electronics Store
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Coffee Shop,Gym,Sandwich Place,Skating Rink,Pool,Dessert Shop,Dim Sum Restaurant


Performing K Means Clustering

In [257]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [258]:
# add clustering labels

neighborhoods_venues_sorted.insert(loc=0, column='Cluster Labels', value=kmeans.labels_)
neighborhoods_venues_sorted.head()


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Adelaide, King, Richmond",Coffee Shop,Bar,Thai Restaurant,Steakhouse,Café,American Restaurant,Bakery,Hotel,Sushi Restaurant,Asian Restaurant
1,2,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,4,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,German Restaurant,Electronics Store
4,2,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Coffee Shop,Gym,Sandwich Place,Skating Rink,Pool,Dessert Shop,Dim Sum Restaurant


In [259]:
neighborhoods_venues_sorted.shape

(99, 12)

In [260]:
for x in range(0,99) :
    neighborhoods_venues_sorted.loc[x,'Cluster Labels']=int(neighborhoods_venues_sorted.loc[x,'Cluster Labels'])

In [261]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Adelaide, King, Richmond",Coffee Shop,Bar,Thai Restaurant,Steakhouse,Café,American Restaurant,Bakery,Hotel,Sushi Restaurant,Asian Restaurant
1,2,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,4,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,German Restaurant,Electronics Store
4,2,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Coffee Shop,Gym,Sandwich Place,Skating Rink,Pool,Dessert Shop,Dim Sum Restaurant


In [262]:
toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,0.0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,2.0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,2.0,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,2.0,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,2.0,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place


In [263]:
toronto_merged.shape

(103, 17)

In [264]:
toronto_merged.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,0.0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,2.0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,2.0,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,2.0,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,2.0,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place


There are certain rows in the Cluster Labels columns that have NaN value which will cause a problem when plotting the folium map, So we assign them to a new cluster 6

In [265]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].fillna(5)

In [266]:
toronto_merged.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,0.0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,2.0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,2.0,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,2.0,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,2.0,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place


# Cluster Labels are of type float after joining. So I'll be converting them to int

In [267]:
list1=toronto_merged['Cluster Labels'].tolist()

In [268]:
list2=[]
for x in list1 :
    list2.append(int(x))

In [269]:
toronto_merged.drop(['Cluster Labels'],axis=1)

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place
5,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242,Toronto-St. Paul's,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,Eglinton-Lawrence,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
7,North York,Don Mills North,M3B,43.745906,-79.352188,Davenport,Gym / Fitness Center,Caribbean Restaurant,Basketball Court,Baseball Field,Japanese Restaurant,Café,Cosmetics Shop,Coworking Space,Falafel Restaurant,Event Space
8,East York,"Woodbine Gardens, Parkview Hill",M4B,43.706397,-79.309937,University-Rosedale,Fast Food Restaurant,Pizza Place,Gastropub,Gym / Fitness Center,Rock Climbing Spot,Café,Breakfast Spot,Bank,Intersection,Athletics & Sports
9,Downtown Toronto,"Ryerson, Garden District",M5B,43.657162,-79.378937,Toronto-Danforth,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Theater,Japanese Restaurant,Restaurant,Italian Restaurant,Ramen Restaurant


In [270]:
toronto_merged['Cluster Labels']=list2

In [271]:
toronto_merged.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,2,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,2,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,2,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,2,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place


In [272]:
toronto_merged.shape

(103, 17)

In [273]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [274]:
c11=toronto_merged

In [275]:
c11=c11.rename(index=str, columns={'1st Most Common Venue': "1", '2nd Most Common Venue': "2",'3rd Most Common Venue':"3",'4th Most Common Venue':'4',
                                   '5th Most Common Venue':'5', '6th Most Common Venue':'6', '7th Most Common Venue' : '7', '8th Most Common Venue' : '8',
                                   '9th Most Common Venue' : '9', '10th Most Common Venue' : '10'})

Renaming all the columns of the toronto_merged dataset so that we can search for our requirements easily

In [276]:
c11.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude,Areas,Cluster Labels,1,2,3,4,5,6,7,8,9,10
0,North York,Parkwoods,M3A,43.753259,-79.329656,NaN,0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,North York,Victoria Village,M4A,43.725882,-79.315572,NaN,2,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Pizza Place,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,Parkdale-High Park,2,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Bank
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,Etobicoke Centre,2,Clothing Store,Event Space,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Gift Shop,Vietnamese Restaurant,Boutique,Accessories Store,Empanada Restaurant
4,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,Etobicoke-Lakeshore,2,Coffee Shop,Gym,Burger Joint,Japanese Restaurant,Diner,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Burrito Place


In [301]:
names = []
bor = []
def searchIndian(c) :
    list11= []
    for i in range(1,11) :
        ind=str(i)
        list11=(c['Neighborhood'].where(c[ind] == 'Indian Restaurant').dropna().tolist())
        if not list11 :
            continue
        for name in list11 :
            names.append(name)
        list11=(c['Borough'].where(c[ind] == 'Indian Restaurant').dropna().tolist())
        if not list11 :
            continue
        for name in list11 :
            bor.append(name)
        

This is a function to search for all the Neighborhoods and Boroughs containing Indian Restaurants in the toronto_merged dataset

In [302]:
searchIndian(c11)

In [303]:
names,bor

(['Thorncliffe Park',
  'Dorset Park, Scarborough Town Centre, Wexford Heights',
  'Woburn',
  'The Annex, North Midtown, Yorkville',
  'Davisville'],
 ['East York',
  'Scarborough',
  'Scarborough',
  'Central Toronto',
  'Central Toronto'])

These are the places containing popular Indian Restaurants. So we should avoid these areas so that our restaurant gets a good customer base

In [280]:
df3 = df2

In [281]:
page='https://en.wikipedia.org/wiki/Demographics_of_Toronto'
wikitable = read_html(page, index_col=0, attrs={"class":"wikitable sortable"})
print("Extracted {num} wikitables".format(num=len(wikitable)))

Extracted 14 wikitables


In [282]:
dff=wikitable[3]

Using the dataset that contains the ethnicity of electoral areas. We clean this dataset so we can use it.

In [283]:
dff.head()

,1,2,3,4,5,6,7
0,,,,,,,
Visible Minorities as % of Population (2016 Census) & Top Ethnic Origins,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Riding (2013 Redistribution),Percentage,Most Populous Ethnic Origin,Percentage,Top Riding for Ethnic Origin (Percentage),Most Populous Visible Minority,Percentage,Top Riding for Visible Minority (Percentage)
Parkdale-High Park,26.2,English,22.3,Irish (20.0); German (9.8); French (8.9),Black,5.3,NaN
Etobicoke Centre,27.1,Italian,15.1,Italian (15.1); Ukrainian (8.1),South Asian,5.9,NaN
Etobicoke-Lakeshore,27.7,English,17.1,NaN,South Asian,5.5,NaN


In [284]:
dff=dff.reset_index()

In [285]:
dff=dff.drop([0],axis=0)

In [286]:
dff=dff.drop([1],axis=0)

In [287]:
dff.head()

,0,1,2,3,4,5,6,7
2,Parkdale-High Park,26.2,English,22.3,Irish (20.0); German (9.8); French (8.9),Black,5.3,NaN
3,Etobicoke Centre,27.1,Italian,15.1,Italian (15.1); Ukrainian (8.1),South Asian,5.9,NaN
4,Etobicoke-Lakeshore,27.7,English,17.1,NaN,South Asian,5.5,NaN
5,Toronto-St. Paul's,28.3,English,18.5,NaN,Black,5.1,NaN
6,Eglinton-Lawrence,31.8,Canadian,14.7,Polish (12.0),Filipino,10.7,NaN


In [288]:
dff=dff.drop([4],axis=1)

In [289]:
dff=dff.drop([5],axis=1)
dff=dff.drop([6],axis=1)
dff=dff.drop([7],axis=1)

In [290]:
dff=dff.drop([1],axis=1)

In [291]:
dff.head()

,0,2,3
2,Parkdale-High Park,English,22.3
3,Etobicoke Centre,Italian,15.1
4,Etobicoke-Lakeshore,English,17.1
5,Toronto-St. Paul's,English,18.5
6,Eglinton-Lawrence,Canadian,14.7


In [292]:
dff=dff.rename(index=str, columns={0: "Neighborhood", 2: "MostEthnic",3:"Percentage"})

In [293]:
dff.head()

,Neighborhood,MostEthnic,Percentage
2,Parkdale-High Park,English,22.3
3,Etobicoke Centre,Italian,15.1
4,Etobicoke-Lakeshore,English,17.1
5,Toronto-St. Paul's,English,18.5
6,Eglinton-Lawrence,Canadian,14.7


Now that we have the areas not to target and the electoral areas with data of ethnicity, we should find out the places that contain Indians. Then, we check if there are any overlapping boroughs. If there are any, then we are supposed to avoid them.

In [294]:
target=dff['Neighborhood'].where(dff['MostEthnic'] == 'East Indian').dropna().tolist()
percent = dff['Percentage'].where(dff['MostEthnic'] == 'East Indian').dropna().tolist()
print(target,percent)

['Don Valley East', 'Scarborough-Guildwood', 'Scarborough-Rouge Park', 'Etobicoke North'] ['10.6', '18.0', '16.7', '22.2']


In [304]:
target_bor=['Scarborough','Scarborough','Scarborough','Etobicoke']

In [306]:
print("Places to start a Indian Restaurant : ")
for borough,per in zip(target_bor,percent) :
    if borough in bor :
        continue
    print(borough,per)

Places to start a Indian Restaurant : 
Etobicoke 22.2


We find the boroughs of the electoral areas. Then we check for overlapping boroughs. Unfortunately, three out of our four electoral areas fell in Scarborough and we're left out with Etobicoke. So Etobicoke by defaults will be the best place to start a restaurant.

In [296]:
print('Best Place to start an Indian Restaurant is : ',dff['Neighborhood'].where(dff['Percentage'] == max(percent)).dropna().tolist())

Best Place to start an Indian Restaurant is :  ['Etobicoke North']


Since, Etobicoke North contains the most Indians, that is the neighborhood that we have to target to get our Restaurant popular. The benefit that also comes with this is that it is in the other side of the city

In [315]:
neighs= df3['Neighborhood'].where(df3['Borough'] == 'Etobicoke').dropna().tolist()
lats = df3['Latitude'].where(df3['Borough'] == 'Etobicoke').dropna().tolist()
longs = df3['Longitude'].where(df3['Borough'] == 'Etobicoke').dropna().tolist()
neighs

['Islington Avenue',
 'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park',
 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
 'Westmount',
 'Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips',
 'Humber Bay Shores, Mimico South, New Toronto',
 'Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown',
 'Alderwood, Long Branch',
 'Northwest',
 'The Kingsway, Montgomery Road, Old Mill North',
 "Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",
 'Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor']

These are the neighbourhoods in Etobicoke

In [335]:
south_lat=43.689879
south_long = -79.5577
#This is the South Boundary of Etobicoke North

In [336]:
new_neighs=[]
for i in range(len(longs)) :
    if (lats[i]>south_lat) :
        new_neighs.append(neighs[i])
print(new_neighs)

['Westmount', 'Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown', 'Northwest']


In [337]:
new_lats=[]
new_longs=[]
for neigh in new_neighs :
    ind= neighs.index(neigh)
    new_lats.append(lats[ind])
    new_longs.append(longs[ind])

These are the neighborhoods that are best suited to construct a Indian Restaurant

In [338]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(new_lats, new_longs, new_neighs):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters